# Manual Labeling & Segmenting

## How to use this notebook:

Use the slider to move through frames.

Click Mark Start to mark the start of a shot.

Move slider to the end frame, enter the shot label in the textbox, then click Mark End.

Repeat for all shots in the video.

Click Save Labels to export all labeled segments as a CSV with frame numbers and corresponding seconds.

In [ ]:
# shuttle-insights/manual_labeling.ipynb

import cv2
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

video_path = '../data/raw/【nice angle】Kunlavut Vitidsarn vs Lu Guang Zu -Sin.mp4'  # Adjust your video path here
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Data storage for labels: shot_id, start_frame, end_frame, label
labels = []
current_shot = {'start': None, 'end': None, 'label': None}
shot_counter = 1

# Widgets
image_widget = widgets.Image(format='jpeg', width=640, height=360)
frame_slider = widgets.IntSlider(min=0, max=total_frames - 1, step=1, description='Frame')
start_button = widgets.Button(description="Mark Start", button_style='success')
end_button = widgets.Button(description="Mark End", button_style='warning')
label_text = widgets.Text(description='Shot Label:')
save_button = widgets.Button(description="Save Labels", button_style='info')
output = widgets.Output()

def update_frame(change):
    frame_no = change['new']
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_no)
    ret, frame = cap.read()
    if ret:
        # Resize for display
        h, w = frame.shape[:2]
        max_w = 640
        scale = max_w / w
        frame = cv2.resize(frame, (max_w, int(h*scale)))
        _, jpeg = cv2.imencode('.jpg', frame)
        image_widget.value = jpeg.tobytes()

def mark_start(b):
    current_frame = frame_slider.value
    current_shot['start'] = current_frame
    with output:
        clear_output(wait=True)
        print(f"Start frame marked at {current_frame}")

def mark_end(b):
    current_frame = frame_slider.value
    if current_shot['start'] is None:
        with output:
            clear_output(wait=True)
            print("Please mark start frame first!")
        return
    if current_frame <= current_shot['start']:
        with output:
            clear_output(wait=True)
            print("End frame must be after start frame!")
        return
    current_shot['end'] = current_frame
    current_shot['label'] = label_text.value.strip()
    if not current_shot['label']:
        with output:
            clear_output(wait=True)
            print("Please enter a shot label!")
        return
    global shot_counter
    labels.append({
        'shot_id': shot_counter,
        'start_frame': current_shot['start'],
        'end_frame': current_shot['end'],
        'label': current_shot['label']
    })
    with output:
        clear_output(wait=True)
        print(f"Saved shot {shot_counter}: {current_shot}")
    shot_counter += 1
    # Reset current shot
    current_shot['start'] = None
    current_shot['end'] = None
    current_shot['label'] = None
    label_text.value = ''

def save_labels(b):
    if not labels:
        with output:
            clear_output(wait=True)
            print("No labels to save!")
        return
    df = pd.DataFrame(labels)
    df['start_sec'] = df['start_frame'] / fps
    df['end_sec'] = df['end_frame'] / fps
    csv_path = '../data/output/manual_labels.csv'
    df.to_csv(csv_path, index=False)
    with output:
        clear_output(wait=True)
        print(f"Labels saved to {csv_path}")
        display(df)

# Link widgets
frame_slider.observe(update_frame, names='value')
start_button.on_click(mark_start)
end_button.on_click(mark_end)
save_button.on_click(save_labels)

# Display interface
display(
    frame_slider,
    image_widget,
    widgets.HBox([start_button, end_button]),
    label_text,
    save_button,
    output,
)

# Initialize first frame
update_frame({'new': 0})